In [48]:
# Run from Model Classes
import importlib
from src import elo_models as EloModels
importlib.reload(EloModels)

# Game Inputs
elo_h_base = 1502  # Home Team
elo_a_base = 1471  # Away Team

# Set Up 538 ELO Model and get vegas line
elo_538 = EloModels.FiveThirtyEightElo(
    home_field_add=55,
    home_field_dist_add=4,
    rest_add=25,
    playoff_mult=1.2
)
vegas_line_538 = elo_538.get_vegas_line(elo_h_base, elo_a_base)
print(f"538's Model Vegas Line = {vegas_line_538}")

# Set Up Johns Model and get vegas line
elo_john = EloModels.JohnElo(
    coeff=0.075,
    exp_ratio=(699/999),
    home_field_add=1.5
)
vegas_line_john = elo_john.get_vegas_line(elo_h_base, elo_a_base)
print(f"John's Model Vegas Line = {vegas_line_john}")

538's Model Vegas Line = -3.44
John's Model Vegas Line = -2.329023025091661
